In [35]:
from collections import namedtuple
from pprint import pprint as pp
 
OpInfo = namedtuple('OpInfo', 'prec assoc')
L, R = 'Left Right'.split()
 
ops = {
 '^': OpInfo(prec=4, assoc=R),
 '*': OpInfo(prec=3, assoc=L),
 '/': OpInfo(prec=3, assoc=L),
 '+': OpInfo(prec=2, assoc=L),
 '-': OpInfo(prec=2, assoc=L),
 '(': OpInfo(prec=9, assoc=L),
 ')': OpInfo(prec=0, assoc=L),
 }

ops_action = {
 '^': lambda a,b: a**b,
 '*': lambda a,b: a*b,
 '/': lambda a,b: a/b,
 '+': lambda a,b: a+b,
 '-': lambda a,b: a-b,
 }
 
NUM, LPAREN, RPAREN = 'NUMBER ( )'.split()
 
 
def get_input(inp = None):
    'Inputs an expression and returns list of (TOKENTYPE, tokenvalue)'
 
    if inp is None:
        inp = input('expression: ')
    tokens = inp.strip().split()
    tokenvals = []
    for token in tokens:
        if token in ops:
            tokenvals.append((token, ops[token]))
        #elif token in (LPAREN, RPAREN):
        #    tokenvals.append((token, token))
        else:    
            tokenvals.append((NUM, token))
    return tokenvals
 
def shunting(tokenvals):
    outq, stack = [], []
    table = ['TOKEN,ACTION,RPN OUTPUT,OP STACK,NOTES'.split(',')]
    for token, val in tokenvals:
        note = action = ''
        if token is NUM:
            action = 'Add number to output'
            outq.append(val)
            table.append( (val, action, ' '.join(outq), ' '.join(s[0] for s in stack), note) )
        elif token in ops:
            t1, (p1, a1) = token, val
            v = t1
            note = 'Pop ops from stack to output' 
            while stack:
                t2, (p2, a2) = stack[-1]
                if (a1 == L and p1 <= p2) or (a1 == R and p1 < p2):
                    if t1 != RPAREN:
                        if t2 != LPAREN:
                            stack.pop()
                            action = '(Pop op)'
                            outq.append(t2)
                        else:    
                            break
                    else:        
                        if t2 != LPAREN:
                            stack.pop()
                            action = '(Pop op)'
                            outq.append(t2)
                        else:    
                            stack.pop()
                            action = '(Pop & discard "(")'
                            table.append( (v, action, ' '.join(outq), ' '.join(s[0] for s in stack), note) )
                            break
                    table.append( (v, action, ' '.join(outq), ' '.join(s[0] for s in stack), note) )
                    v = note = ''
                else:
                    note = ''
                    break
                note = '' 
            note = '' 
            if t1 != RPAREN:
                stack.append((token, val))
                action = 'Push op token to stack'
            else:
                action = 'Discard ")"'
            table.append( (v, action, ' '.join(outq), ' '.join(s[0] for s in stack), note) )
    note = 'Drain stack to output'
    while stack:
        v = ''
        t2, (p2, a2) = stack[-1]
        action = '(Pop op)'
        stack.pop()
        outq.append(t2)
        table.append( (v, action, ' '.join(outq), ' '.join(s[0] for s in stack), note) )
        v = note = ''
    return table

In [49]:
infix = '3 + 4 * 2 ^ ( 1 + 2 )'
print( 'For infix expression: %r\n' % infix )
print(get_input(infix))
rp = shunting(get_input(infix))
maxcolwidths = [len(max(x, key=len)) for x in zip(*rp)]
row = rp[0]
print( ' '.join('{cell:^{width}}'.format(width=width, cell=cell) for (width, cell) in zip(maxcolwidths, row)))
for row in rp[1:]:
    print( ' '.join('{cell:<{width}}'.format(width=width, cell=cell) for (width, cell) in zip(maxcolwidths, row)))
 
print('\n The final output RPN is: %r' % rp[-1][2])

For infix expression: '3 + 4 * 2 ^ ( 1 + 2 )'

[('NUMBER', '3'), ('+', OpInfo(prec=2, assoc='Left')), ('NUMBER', '4'), ('*', OpInfo(prec=3, assoc='Left')), ('NUMBER', '2'), ('^', OpInfo(prec=4, assoc='Right')), ('(', OpInfo(prec=9, assoc='Left')), ('NUMBER', '1'), ('+', OpInfo(prec=2, assoc='Left')), ('NUMBER', '2'), (')', OpInfo(prec=0, assoc='Left'))]
TOKEN         ACTION            RPN OUTPUT     OP STACK             NOTES            
3     Add number to output   3                                                       
+     Push op token to stack 3                 +                                     
4     Add number to output   3 4               +                                     
*     Push op token to stack 3 4               + *                                   
2     Add number to output   3 4 2             + *                                   
^     Push op token to stack 3 4 2             + * ^                                 
(     Push op token to stack 3 4 2        

In [46]:
postfix = rp[-1][2].split(" ")
postfix

['3', '4', '2', '1', '2', '+', '^', '*', '+']

In [47]:
stack = []
for token in postfix:
    if token in ops:
        operand_1 = stack.pop()
        operand_2 = stack.pop()
        print(str(operand_1) + token + str(operand_2))
        result = ops_action[token](float(operand_1), float(operand_2))
        #result = eval(str(operand_1) + token + str(operand_2))
        print("temp result: " + str(result))
        stack.append(result)
    else:
        stack.append(token)
result = stack.pop()
print("Result: " + str(result))

2+1
temp result: 3.0
3.0^2
temp result: 9.0
9.0*4
temp result: 36.0
36.0+3
temp result: 39.0
Result: 39.0
